In [1]:
from pathlib import Path
import sys

In [2]:
PROJECT_PATH = Path().resolve().parents[0]
sys.path.append(str(PROJECT_PATH))

In [3]:
from object.class_file import BatchConfig
from pathlib import Path
from object.class_file import StrategyParams
from backtesting.global_loop import run_global_ranking_walkforward

In [4]:

BASE_DATA_PATH = PROJECT_PATH / "data" / "raw"
SCANNER_DATA_PATH = BASE_DATA_PATH / "d1"
DATA_PATH = PROJECT_PATH / "data" / "raw" / "d1"

In [5]:
SELECTED_UNIVERSE = ["sweden"]

In [6]:
cfg = BatchConfig(
        data_path = PROJECT_PATH / "data" / "raw" / "d1",
        monthly_universe_path = PROJECT_PATH / "data" / "universe" ,
        out_dir = PROJECT_PATH / "data" / "backtests" / "global_ranking",
        universe_name="GLOBAL",
        timeframe="Daily",
        warmup_extra=50,
        equal_weight=True,
    )

In [7]:
z_entry = 2
z_exit = 4
z_stop = 0
z_window = 60
wf_train = 120
fees = 0.0002
beta_mode = "monthly"
N = 20
K = 5

params = StrategyParams(
        z_entry=float(z_entry),
        z_exit=float(z_exit),
        z_stop=float(z_stop),
        z_window=int(z_window),
        wf_train=int(wf_train),
        wf_test=0,
        fees=float(fees),
        beta_mode=str(beta_mode),
        top_n_candidates= int(N),
        max_positions= int(K)
)

In [8]:
res = run_global_ranking_walkforward(
    cfg=cfg,
    params=params,
    universes=list(SELECTED_UNIVERSE),
)

if not res:
    print("No results.")
else:
    print("DONE")
    print("Stats:", res["stats"])
    print("Trades:", len(res["trades"]))
    print("Equity rows:", len(res["equity"]))

DONE
Stats: {'Final Equity': 1.29, 'CAGR': 0.08, 'Sharpe': 0.75, 'Max Drawdown': -0.11, 'Nb Trades': 671}
Trades: 671
Equity rows: 807


In [9]:
res["trades"]

,trade_month,pair_id,asset_1,asset_2,side,beta,entry_datetime,entry_z,entry_spread,exit_datetime,exit_z,exit_spread,pnl_spread,reason,duration_days
0,2021-01,SKF_AB__SVENSKA_CELLULOSA_AB__2021-01,SKF_AB,SVENSKA_CELLULOSA_AB,SHORT_SPREAD,0.821209,2021-01-12,2.382796,-0.064063,2021-01-13,2.753662,-0.054652,-0.009411,TP,1
1,2021-01,SKF_AB__SVENSKA_CELLULOSA_AB__2021-01,SKF_AB,SVENSKA_CELLULOSA_AB,SHORT_SPREAD,0.821209,2021-01-13,2.753662,-0.054652,2021-01-14,3.173254,-0.041545,-0.013108,TP,1
2,2021-01,SKF_AB__SVENSKA_CELLULOSA_AB__2021-01,SKF_AB,SVENSKA_CELLULOSA_AB,SHORT_SPREAD,0.821209,2021-01-14,3.173254,-0.041545,2021-01-15,2.966883,-0.040501,-0.001044,TP,1
3,2021-01,SKF_AB__SVENSKA_CELLULOSA_AB__2021-01,SKF_AB,SVENSKA_CELLULOSA_AB,SHORT_SPREAD,0.821209,2021-01-15,2.966883,-0.040501,2021-01-16,2.761019,-0.040501,-0.000000,TP,1
4,2021-01,SKF_AB__SVENSKA_CELLULOSA_AB__2021-01,SKF_AB,SVENSKA_CELLULOSA_AB,SHORT_SPREAD,0.821209,2021-01-16,2.761019,-0.040501,2021-01-17,2.555929,-0.040501,-0.000000,TP,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,2025-11,ALFA_LAVAL_AB__SKANSKA_AB__2025-11,ALFA_LAVAL_AB,SKANSKA_AB,SHORT_SPREAD,0.520546,2025-11-12,2.293311,0.840365,2025-11-13,1.569029,0.829641,0.010725,TP,1
667,2025-12,ALFA_LAVAL_AB__SKANSKA_AB__2025-12,ALFA_LAVAL_AB,SKANSKA_AB,SHORT_SPREAD,0.495160,2025-12-04,2.820559,0.875855,2025-12-05,2.438007,0.871688,0.004167,TP,1
668,2025-12,ALFA_LAVAL_AB__SKANSKA_AB__2025-12,ALFA_LAVAL_AB,SKANSKA_AB,SHORT_SPREAD,0.495160,2025-12-05,2.438007,0.871688,2025-12-06,2.290794,0.871688,-0.000000,TP,1
669,2025-12,ALFA_LAVAL_AB__SKANSKA_AB__2025-12,ALFA_LAVAL_AB,SKANSKA_AB,SHORT_SPREAD,0.495160,2025-12-06,2.290794,0.871688,2025-12-07,2.168471,0.871688,-0.000000,TP,1


In [10]:
res["trades"].groupby('reason').agg({'pair_id': 'count', 'pnl_spread': ['sum', 'median', 'std']}).round(4)

pair_id pnl_spread                
         count        sum  median     std
reason                                   
SL           3    -0.0631 -0.0073  0.0277
TP         668     0.4745  0.0000  0.0123

In [11]:
res["trades"].groupby('trade_month').agg({'pair_id': 'count', 'pnl_spread': ['sum', 'median', 'std']}).round(4)

pair_id pnl_spread                
              count        sum  median     std
trade_month                                   
2021-01          21    -0.0192  0.0000  0.0100
2021-02          42    -0.0644 -0.0000  0.0118
2021-08           4     0.0180  0.0024  0.0071
2021-09          55    -0.0837  0.0000  0.0199
2022-05          20     0.1468  0.0002  0.0179
2022-11          12    -0.0019 -0.0000  0.0186
2022-12          16    -0.0375  0.0000  0.0079
2023-01          45     0.0422  0.0000  0.0150
2023-02          16     0.0240 -0.0000  0.0089
2023-09          56     0.0572 -0.0000  0.0108
2023-10          38    -0.0778  0.0000  0.0134
2023-11          35    -0.0169 -0.0000  0.0120
2023-12          31     0.0222 -0.0000  0.0051
2024-03          26     0.0789  0.0000  0.0192
2024-04          19    -0.0198  0.0000  0.0070
2024-05          14     0.0147  0.0000  0.0139
2024-06          27     0.0871  0.0000  0.0143
2024-10          45     0.1226 -0.0000  0.0089
2024-12          22     0.0194  0.0000  0.0068
2025-01          40    -0.0468 -0.0000  0.0068
2025-02          10     0.0100  0.0000  0.0074
2025-05           8    -0.0160  0.0000  0.0056
2025-06          35     0.0494  0.0000  0.0062
2025-07          11     0.0790  0.0000  0.0185
2025-09          14    -0.0051  0.0002  0.0066
2025-11           5     0.0203  0.0001  0.0055
2025-12           4     0.0087  0.0021  0.0025

In [12]:
pnl = res["trades"]["pnl_spread"].dropna()

pnl.describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])

count    671.000000
mean       0.000613
std        0.012429
min       -0.065224
1%        -0.036310
5%        -0.018620
25%       -0.001912
50%        0.000000
75%        0.004608
95%        0.019509
99%        0.038118
max        0.059489
Name: pnl_spread, dtype: float64

In [13]:
print("Skew:", pnl.skew())
print("Kurtosis:", pnl.kurtosis())
print("Win rate:", (pnl > 0).mean())

Skew: 0.019278790584408845
Kurtosis: 5.607254317008744
Win rate: 0.36065573770491804
